## LIBRERIAS

In [277]:
import pandas as pd
import numpy as np
import plotly_express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score, roc_auc_score, roc_curve, classification_report


In [278]:
#Logistic regression
#Random forest
# KNN

#Lightgbm
# CatBoost
# Red Neuronal

In [279]:
try:
    data_contract = pd.read_csv("final_provider/contract.csv")
    data_internet = pd.read_csv("final_provider/internet.csv")
    data_personal = pd.read_csv("final_provider/personal.csv")
    data_phone    = pd.read_csv("final_provider/phone.csv")
except:
    data_contract = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/contract.csv")
    data_internet = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/internet.csv")
    data_personal = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/personal.csv")
    data_phone    = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com//datasets/final_provider/phone.csv")



In [280]:
data_contract.set_index('customerID', inplace=True)
data_internet.set_index('customerID', inplace=True)
data_personal.set_index('customerID', inplace=True)
data_phone.set_index('customerID', inplace=True)


In [281]:
#juntar todas las tablas
full_data = data_contract.join([data_internet, data_personal, data_phone])


In [282]:
full_data["BeginDate"] = pd.to_datetime(full_data["BeginDate"], errors='coerce')
full_data["EndDate"] = pd.to_datetime(full_data["EndDate"], errors='coerce')

full_data['BeginDate_Year'] = full_data['BeginDate'].dt.year
full_data['BeginDate_Month'] = full_data['BeginDate'].dt.month

full_data['BeginDate_Year'] = full_data['BeginDate_Year'].astype("str")
full_data['BeginDate_Month'] = full_data['BeginDate_Month'].astype("str")

# Creación de la columna objetivo (target)
full_data["isActive"] = full_data["EndDate"].apply(
    lambda x: 0 if pd.isna(x) else 1
)

full_data = full_data.drop(columns=["BeginDate", "EndDate"], axis =1)

# Relleno de valores ausentes, al observar los datos faltantes es muy probable que todos sean por el tipo de plan contratado, no existen datos porque no han contratado un plan que pueda generar datos en esas características
full_data.fillna("No data", inplace=True)

# Reemplazamos strings vacios por ceros
full_data.replace({"MonthlyCharges": {" ": 0}, "TotalCharges": {" ": 0}}, inplace=True)






C:\Users\armod\AppData\Local\Temp\ipykernel_17608\1594772674.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [283]:
full_data.sample(10)



,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,gender,SeniorCitizen,Partner,Dependents,MultipleLines,BeginDate_Year,BeginDate_Month,isActive
customerID,,,,,,,,,,,,,,,,,,,,
8048-DSDFQ,Month-to-month,No,Mailed check,20.20,20.2,No data,No data,No data,No data,No data,No data,No data,Male,0,No,No,No,2020,1,0
4884-ZTHVF,Month-to-month,Yes,Electronic check,87.10,713.6,Fiber optic,No,Yes,No,No,Yes,No,Female,1,No,No,No,2019,6,0
0947-MUGVO,Month-to-month,Yes,Electronic check,73.65,1463.5,Fiber optic,No,Yes,No,No,No,No,Male,1,Yes,No,No,2018,4,1
8313-AFGBW,Two year,No,Electronic check,73.60,3522.65,DSL,No,Yes,Yes,No,Yes,Yes,Male,0,Yes,No,No,2016,3,0
0402-CQAJN,Month-to-month,Yes,Bank transfer (automatic),101.90,3545.35,Fiber optic,No,Yes,Yes,No,Yes,Yes,Female,0,No,No,Yes,2016,10,1
2122-YWVYA,Month-to-month,Yes,Electronic check,75.40,1380.4,Fiber optic,Yes,No,No,No,No,No,Female,0,No,No,No,2018,8,0
8109-YUOHE,One year,No,Mailed check,56.10,946.95,DSL,Yes,Yes,No,No,No,No,Male,0,No,No,No,2018,9,0
3312-UUMZW,Month-to-month,Yes,Credit card (automatic),98.85,3145.9,Fiber optic,No,Yes,No,No,Yes,Yes,Male,0,Yes,No,Yes,2017,6,0
7299-GNVPL,Month-to-month,No,Mailed check,84.40,1936.85,Fiber optic,Yes,Yes,No,Yes,No,No,Female,0,Yes,Yes,No,2018,3,0


In [284]:
# for col in full_data:
#     print(full_data[col].unique())


## MODELOS BASICOS

#### LOGISTIC REGRESSION

In [285]:
cols_categ = (
    [
        "Type",
        "PaperlessBilling",
        "PaymentMethod",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies",
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents",
        "MultipleLines",
        "isActive",
        "BeginDate_Year",
        "BeginDate_Month",
        "EndDate_Year",
        "EndDate_Month",
    ],
)
numeric_columns = ["MonthlyCharges", "TotalCharges"]

In [286]:
def scaler_ohe(df):
    scaler = StandardScaler()
    scaler.fit(df[numeric_columns])
    data_scaled = scaler.transform(df[numeric_columns])
    df[numeric_columns]= data_scaled
    df = pd.get_dummies(df, drop_first=True)
    
    return df

def find_scores(features_test, target_test, predictions, model):

    print(classification_report(target_test, predictions))
    print("Área bajo la curva ROC:", roc_auc_score(target_test, predictions))

    probab_test = model.predict_proba(features_test)
    probab_one_test = probab_test[:, 1]

    fpr, tpr, thresholds = roc_curve(target_test, probab_one_test)  

    #Creamos un df con los datos para graficar
    roc_data = {"FPR": fpr, "TPR": tpr}
    df_roc = pd.DataFrame(roc_data)

    # Trazar la curva ROC con Plotly Express
    fig = px.line(
        df_roc,
        x="FPR",
        y="TPR",
        labels={"FPR": "Tasa de falsos positivos", "TPR": "Tasa de verdaderos positivos"},
        title="Curva ROC",
    )

    # Agregar una línea diagonal que representa el modelo aleatorio
    fig.add_shape(type="line", line=dict(dash="dash"), x0=0, x1=1, y0=0, y1=1)
    fig.update_layout(
        width=800,  
        height=450
    )
    # Mostrar la figura
    fig.show()

In [287]:
y = full_data["isActive"]
X = full_data.drop(columns="isActive")
X = scaler_ohe(X)
X_train,X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=999)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=999)
y.value_counts()

isActive
0    5174
1    1869
Name: count, dtype: int64

In [288]:
model_logistreg = LogisticRegression(class_weight="balanced")
model_logistreg.fit(X_train, y_train)
predictions = model_logistreg.predict(X_test)


find_scores(X_test, y_test, predictions, model_logistreg)

              precision    recall  f1-score   support

           0       0.93      0.73      0.82       844
           1       0.51      0.82      0.63       283

    accuracy                           0.76      1127
   macro avg       0.72      0.78      0.72      1127
weighted avg       0.82      0.76      0.77      1127

Área bajo la curva ROC: 0.7789593555842111


#### RANDOM FOREST CLASSIFIER

In [289]:
model_rf = RandomForestClassifier(class_weight="balanced")
model_rf.fit(X_train,y_train)
predictions = model_rf.predict(X_test)


find_scores(X_test,y_test, predictions, model_rf)

              precision    recall  f1-score   support

           0       0.87      0.94      0.90       844
           1       0.76      0.57      0.65       283

    accuracy                           0.85      1127
   macro avg       0.81      0.75      0.78      1127
weighted avg       0.84      0.85      0.84      1127

Área bajo la curva ROC: 0.7542390266776078


#### KNN CLASSIFIER

In [290]:
model_knn = KNeighborsClassifier(n_neighbors=5)  

# Entrenar el modelo
model_knn.fit(X_train, y_train)
predictions = model_knn.predict(X_test)

find_scores(X_test, y_test, predictions, model_knn)

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       844
           1       0.58      0.59      0.59       283

    accuracy                           0.79      1127
   macro avg       0.72      0.72      0.72      1127
weighted avg       0.79      0.79      0.79      1127

Área bajo la curva ROC: 0.7245553732018154


#### LIGHTGBM